In [ ]:
import glob
import os
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
from sklearn.metrics import precision_recall_fscore_support
from datetime import datetime

In [86]:
# データ読み込み
import urbansound8k_loader as dataset

#parent_dir = 'UrbanSound8K/audio/'
#tr_features, tr_labels = dataset.load_urbansound8k(parent_dir, ['fold1', 'fold2'])
#ts_features, ts_labels = dataset.load_urbansound8k(parent_dir, ['fold3'])

%time tr_features, tr_labels, ts_features, ts_labels = dataset.load_from_npy_files()

if (len(tr_features) !=  len(tr_labels)):
    print('WARN: invalid # of training data. features=' + str(len(tr_features)) + ', labels=' + str(len(tr_labels)))
elif (len(ts_features) !=  len(ts_labels)):
    print('WARN: invalid # of tast data. features=' + str(len(ts_features)) + ', labels=' + str(len(ts_labels)))
elif (len(tr_features) == 0 || len(ts_features) == 0):
    print('WARN: no data.')
else: 
    print('loaded successfully. # of train data=' + str(len(tr_features)) + ', # of test data=' + str(len(ts_features)))


SyntaxError: invalid syntax (<ipython-input-86-b48159028c01>, line 14)

In [64]:
training_epochs = 5000
n_dim = tr_features.shape[1]
n_classes = 10
n_hidden_units_one = 280 
n_hidden_units_two = 300
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.01
log_dir = './log'

In [65]:
# モデル構築、学習、評価 by keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import keras.callbacks
import keras.backend.tensorflow_backend as KTF

log_filepath = './keras/log'
batch_size = 128
old_session = KTF.get_session()

with tf.Graph().as_default():
    session = tf.Session('')
    KTF.set_session(session)
    KTF.set_learning_phase(1)
    
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(n_dim,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)
    cbks = [tb_cb]
    
    print ('fit start: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    history = model.fit(tr_features, tr_labels,
                        batch_size=batch_size,
                        epochs=training_epochs,
                        verbose=1,
                        callbacks=cbks, 
                        validation_data=(ts_features, ts_labels))

    print ('evaluate start: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))
    score = model.evaluate(ts_features, ts_labels, verbose=0)
    print('Test accuracy:', score[1])
    print ('finished: ', datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

KTF.set_session(old_session)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               99328     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 367,114
Trainable params: 367,114
Non-trainable params: 0
_________________________________________________________________
fit start:  2017/07/02 13:04:05
Train on 1761 samples, validate on 925 samples
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal;

1761/1761 [==============================] - 1s - loss: 0.7700 - acc: 0.9438 - val_loss: 6.6969 - val_acc: 0.3503
Epoch 53/5000
1761/1761 [==============================] - 0s - loss: 0.7695 - acc: 0.9432 - val_loss: 6.6011 - val_acc: 0.3719
Epoch 54/5000
1761/1761 [==============================] - 0s - loss: 0.7155 - acc: 0.9461 - val_loss: 6.8667 - val_acc: 0.3578
Epoch 55/5000
1761/1761 [==============================] - 0s - loss: 0.6893 - acc: 0.9546 - val_loss: 7.2131 - val_acc: 0.3395
Epoch 56/5000
1761/1761 [==============================] - 0s - loss: 0.7097 - acc: 0.9500 - val_loss: 6.8899 - val_acc: 0.3978
Epoch 57/5000
1761/1761 [==============================] - 0s - loss: 0.7084 - acc: 0.9466 - val_loss: 7.0689 - val_acc: 0.3762
Epoch 58/5000
1761/1761 [==============================] - 0s - loss: 0.7216 - acc: 0.9506 - val_loss: 6.5916 - val_acc: 0.4000
Epoch 59/5000
1761/1761 [==============================] - 0s - loss: 0.7141 - acc: 0.9483 - val_loss: 6.6832 - val_ac

1761/1761 [==============================] - 0s - loss: 0.6702 - acc: 0.9574 - val_loss: 7.6541 - val_acc: 0.3892
Epoch 116/5000
1761/1761 [==============================] - 0s - loss: 0.6997 - acc: 0.9512 - val_loss: 7.6609 - val_acc: 0.3578
Epoch 117/5000
1761/1761 [==============================] - 0s - loss: 0.6766 - acc: 0.9546 - val_loss: 7.5440 - val_acc: 0.3795
Epoch 118/5000
1761/1761 [==============================] - 0s - loss: 0.6913 - acc: 0.9512 - val_loss: 7.4724 - val_acc: 0.3914
Epoch 119/5000
1761/1761 [==============================] - 0s - loss: 0.6859 - acc: 0.9534 - val_loss: 7.9423 - val_acc: 0.3622
Epoch 120/5000
1761/1761 [==============================] - 0s - loss: 0.6642 - acc: 0.9580 - val_loss: 7.9390 - val_acc: 0.3546
Epoch 121/5000
1761/1761 [==============================] - 1s - loss: 0.6724 - acc: 0.9534 - val_loss: 7.7752 - val_acc: 0.3751
Epoch 122/5000
1761/1761 [==============================] - 0s - loss: 0.6704 - acc: 0.9580 - val_loss: 7.4862 -

1761/1761 [==============================] - 1s - loss: 0.6855 - acc: 0.9534 - val_loss: 8.1002 - val_acc: 0.3676
Epoch 179/5000
1761/1761 [==============================] - 1s - loss: 0.6620 - acc: 0.9568 - val_loss: 8.1184 - val_acc: 0.3665
Epoch 180/5000
1761/1761 [==============================] - 0s - loss: 0.6741 - acc: 0.9557 - val_loss: 7.8374 - val_acc: 0.4054
Epoch 181/5000
1761/1761 [==============================] - 0s - loss: 0.6600 - acc: 0.9591 - val_loss: 8.0611 - val_acc: 0.3957
Epoch 182/5000
1761/1761 [==============================] - 0s - loss: 0.6620 - acc: 0.9585 - val_loss: 8.0563 - val_acc: 0.3805
Epoch 183/5000
1761/1761 [==============================] - 0s - loss: 0.6648 - acc: 0.9585 - val_loss: 8.2629 - val_acc: 0.3795
Epoch 184/5000
1761/1761 [==============================] - 0s - loss: 0.6673 - acc: 0.9563 - val_loss: 8.4180 - val_acc: 0.3589
Epoch 185/5000
1761/1761 [==============================] - 0s - loss: 0.6767 - acc: 0.9534 - val_loss: 8.0368 -

...

1761/1761 [==============================] - 1s - loss: 7.9769e-05 - acc: 1.0000 - val_loss: 8.9041 - val_acc: 0.4314
Epoch 4702/5000
1761/1761 [==============================] - 1s - loss: 0.0064 - acc: 0.9989 - val_loss: 8.8970 - val_acc: 0.4281
Epoch 4703/5000
1761/1761 [==============================] - 1s - loss: 0.0330 - acc: 0.9972 - val_loss: 9.2376 - val_acc: 0.4054
Epoch 4704/5000
1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 8.9861 - val_acc: 0.4173
Epoch 4705/5000
1761/1761 [==============================] - 1s - loss: 0.0183 - acc: 0.9989 - val_loss: 9.0583 - val_acc: 0.4097
Epoch 4706/5000
1761/1761 [==============================] - 1s - loss: 0.0286 - acc: 0.9977 - val_loss: 9.4854 - val_acc: 0.3935
Epoch 4707/5000
1761/1761 [==============================] - 1s - loss: 0.0458 - acc: 0.9972 - val_loss: 9.4300 - val_acc: 0.3914
Epoch 4708/5000
1761/1761 [==============================] - 1s - loss: 0.0183 - acc: 0.9989 - val_los

1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 8.8038 - val_acc: 0.4292
Epoch 4764/5000
1761/1761 [==============================] - 1s - loss: 0.0114 - acc: 0.9989 - val_loss: 9.1928 - val_acc: 0.4108
Epoch 4765/5000
1761/1761 [==============================] - 1s - loss: 0.0139 - acc: 0.9989 - val_loss: 9.0230 - val_acc: 0.4173
Epoch 4766/5000
1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 9.2144 - val_acc: 0.4086
Epoch 4767/5000
1761/1761 [==============================] - 1s - loss: 0.0149 - acc: 0.9983 - val_loss: 8.9968 - val_acc: 0.4184
Epoch 4768/5000
1761/1761 [==============================] - 1s - loss: 0.0139 - acc: 0.9983 - val_loss: 9.0879 - val_acc: 0.4173
Epoch 4769/5000
1761/1761 [==============================] - 1s - loss: 0.0078 - acc: 0.9994 - val_loss: 9.0873 - val_acc: 0.4173
Epoch 4770/5000
1761/1761 [==============================] - 1s - loss: 0.0178 - acc: 0.9983 - val_loss: 8

1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 9.0863 - val_acc: 0.4238
Epoch 4826/5000
1761/1761 [==============================] - 1s - loss: 0.0093 - acc: 0.9994 - val_loss: 8.8321 - val_acc: 0.4346
Epoch 4827/5000
1761/1761 [==============================] - 1s - loss: 0.0038 - acc: 0.9994 - val_loss: 8.8204 - val_acc: 0.4389
Epoch 4828/5000
1761/1761 [==============================] - 1s - loss: 0.0183 - acc: 0.9989 - val_loss: 8.7663 - val_acc: 0.4368
Epoch 4829/5000
1761/1761 [==============================] - 1s - loss: 0.0183 - acc: 0.9989 - val_loss: 9.0128 - val_acc: 0.4249
Epoch 4830/5000
1761/1761 [==============================] - 1s - loss: 0.0164 - acc: 0.9983 - val_loss: 9.0021 - val_acc: 0.4249
Epoch 4831/5000
1761/1761 [==============================] - 1s - loss: 1.3761e-04 - acc: 1.0000 - val_loss: 9.2339 - val_acc: 0.4119
Epoch 4832/5000
1761/1761 [==============================] - 1s - loss: 0.0352 - acc: 0.9977 - val_los

1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 9.6145 - val_acc: 0.3859
Epoch 4888/5000
1761/1761 [==============================] - 1s - loss: 0.0034 - acc: 0.9994 - val_loss: 9.3492 - val_acc: 0.3978
Epoch 4889/5000
1761/1761 [==============================] - 1s - loss: 2.6649e-05 - acc: 1.0000 - val_loss: 9.3933 - val_acc: 0.3914
Epoch 4890/5000
1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 9.3998 - val_acc: 0.3935
Epoch 4891/5000
1761/1761 [==============================] - 1s - loss: 0.0063 - acc: 0.9983 - val_loss: 9.1161 - val_acc: 0.4086
Epoch 4892/5000
1761/1761 [==============================] - 1s - loss: 0.0276 - acc: 0.9977 - val_loss: 9.2587 - val_acc: 0.4076
Epoch 4893/5000
1761/1761 [==============================] - 1s - loss: 0.0093 - acc: 0.9994 - val_loss: 9.2977 - val_acc: 0.4130
Epoch 4894/5000
1761/1761 [==============================] - 0s - loss: 5.9872e-07 - acc: 1.0000 - val

1761/1761 [==============================] - 1s - loss: 1.2138e-07 - acc: 1.0000 - val_loss: 8.7312 - val_acc: 0.4368
Epoch 4950/5000
1761/1761 [==============================] - 1s - loss: 0.0021 - acc: 0.9989 - val_loss: 8.8934 - val_acc: 0.4303
Epoch 4951/5000
1761/1761 [==============================] - 1s - loss: 0.0092 - acc: 0.9994 - val_loss: 9.0180 - val_acc: 0.4249
Epoch 4952/5000
1761/1761 [==============================] - 0s - loss: 3.3039e-04 - acc: 1.0000 - val_loss: 8.8811 - val_acc: 0.4259
Epoch 4953/5000
1761/1761 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 9.0705 - val_acc: 0.4184
Epoch 4954/5000
1761/1761 [==============================] - 1s - loss: 0.0247 - acc: 0.9983 - val_loss: 9.1350 - val_acc: 0.4119
Epoch 4955/5000
1761/1761 [==============================] - 1s - loss: 0.0136 - acc: 0.9989 - val_loss: 9.0214 - val_acc: 0.4205
Epoch 4956/5000
1761/1761 [==============================] - 1s - loss: 0.0183 - acc: 0.9989 -

In [71]:
type(tr_features)

numpy.ndarray

In [80]:
np.save('tr_features.npy', tr_features)
np.save('tr_labels.npy', tr_labels)
np.save('ts_features.npy', ts_features)
np.save('ts_labels.npy', ts_labels)